In [3]:
# github.com/minrk/findspark
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName="Spark1")

In [4]:
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.cm as cm
def mandelbrot(x, y):
    z = c = complex(x, y)
    iteration = 0
    max_iteration = 511  # arbitrary cutoff
    while abs(z) < 2 and iteration < max_iteration:
        z = z * z + c
        iteration += 1
    return iteration

def sum_values_for_partitions(rdd):
    'Returns (as an RDD) the sum of V for each partition of a (K, V) RDD'
    # note that the function passed to mapPartitions should return a sequence,
    # not a value.
    return rdd.mapPartitions(lambda part: [sum(V for K, V in part)])

def draw_image(rdd):
    '''Given a (K, V) RDD with K = (I, J) and V = count,
    display an image of count at each I, J'''

    data = rdd.collect()
    I = np.array([d[0][0] for d in data])
    J = np.array([d[0][1] for d in data])
    C = np.array([d[1] for d in data])
    im = np.zeros((I.max() + 1, J.max() + 1))
    im[I, J] = np.log(C + 1)  # log intensity makes it easier to see levels
    plt.imshow(im, cmap=cm.gray)
    plt.show()



In [ ]:
#image is 2000 pixels x 2000 pixels
dtype=[('a','<i4'),('b','<i4')]
data_matrix = np.zeros((2000,2000),dtype=[('i','i4'),('j','i4'),('x','<f4'),('y','<f4')])
for i in range(2000):
    for j in range(2000):
        x = j / 500.0 - 2
        y = i / 500.0 - 2
        data_matrix[i][j] = (i, j, x, y)
data_rdd = sc.parallelize(data_matrix,100)
rdd = data_rdd.flatMap(lambda x: x)

iters = rdd.map(lambda (i,j,x,y): ((i,j),mandelbrot(x,y)))
num_per_part = sum_values_for_partitions(iters)
#draw_image(iters)
plt.hist(num_per_part.collect())
plt.savefig()